<a href="https://colab.research.google.com/github/as2c/MLP---Cosmology/blob/main/Data2/density_contrast_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
!git clone https://github.com/jveitchmichaelis/pygadgetreader

fatal: destination path 'pygadgetreader' already exists and is not an empty directory.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/pygadgetreader
!python setup.py build
!python setup.py install

/content/pygadgetreader
running build
running build_py
running install
running bdist_egg
running egg_info
writing pyGadgetReader.egg-info/PKG-INFO
writing dependency_links to pyGadgetReader.egg-info/dependency_links.txt
writing requirements to pyGadgetReader.egg-info/requires.txt
writing top-level names to pyGadgetReader.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'pyGadgetReader.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pygadgetreader
copying build/lib/pygadgetreader/__init__.py -> build/bdist.linux-x86_64/egg/pygadgetreader
creating build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readrockstar.py -> build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readpstar.py -> build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readgadget.py -> build/bdist.linux-x8

In [4]:
from pygadgetreader import *

In [5]:
# vel = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','vel','dm')
# vx = vel[:,0].tolist()
# vy = vel[:,1].tolist()
# vz = vel[:,2].tolist()
# mass = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','mass','dm').tolist()
# particle_id = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pid','dm').tolist()
# position = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pos','dm')
# x = position[:,0].tolist()
# y = position[:,1].tolist()
# z = position[:,2].tolist()
# data = {'particle_id': particle_id,
#         'x': x,
#         'y': y,
#         'z': z,
#         'vx': vx,
#         'vy': vy,
#         'vz': vz,
#         'mass': mass}
# df = pd.DataFrame(data)from pygadgetreader import *

# seed = int(input("Enter the seed:"))
halopath = '/content/drive/MyDrive/ML/Project/halos_0'
initial_snap_path = '/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm'

# print("The seed is ", seed, " Now the code just started")
#Reading the halo particles 

particles = readrockstar(halopath,'particles')
halo_particles = pd.DataFrame()
halo_particles['id'] = particles[:,0]
halo_particles['haloID'] = particles[:,1]

#Reading the halo masses
halo_num = pd.DataFrame() 
halos = readrockstar(halopath, 'halos') 
halo_num['haloID'] = halos['id']
halo_num['np'] = halos['num_p']

#a dataframe containing the subhalo id and sub halo mass for each particle
halo_particles = pd.merge(halo_particles, halo_num , right_on = 'haloID', left_on = 'haloID', how = 'outer')

print(halo_particles.describe())

                 id        haloID            np
count  2.214619e+06  2.214619e+06  2.214619e+06
mean   8.946561e+06  6.562720e+02  4.008056e+03
std    4.967033e+06  3.655899e+02  5.164701e+03
min    1.000000e+01  0.000000e+00  2.020000e+02
25%    4.549009e+06  3.370000e+02  1.218000e+03
50%    9.372965e+06  6.610000e+02  2.190000e+03
75%    1.337186e+07  9.760000e+02  4.205000e+03
max    1.677717e+07  1.272000e+03  3.125200e+04


In [6]:
particle_data = pd.DataFrame()

particle_data['id'] = np.array(readsnap(initial_snap_path, 'pid', 'dm'))

particle_data['x'] = np.array(readsnap(initial_snap_path, 'pos', 'dm'))[:,0]
particle_data['y'] = np.array(readsnap(initial_snap_path, 'pos', 'dm'))[:,1]
particle_data['z'] = np.array(readsnap(initial_snap_path, 'pos', 'dm'))[:,2]

particle_data['vx'] = np.array(readsnap(initial_snap_path, 'vel', 'dm'))[:,0]
particle_data['vy'] = np.array(readsnap(initial_snap_path, 'vel', 'dm'))[:,1]
particle_data['vz'] = np.array(readsnap(initial_snap_path, 'vel', 'dm'))[:,2]

Returning DM    Particle IDs
Returning DM    Positions
Returning DM    Positions
Returning DM    Positions
Returning DM    Velocities
Returning DM    Velocities
Returning DM    Velocities


In [58]:
data = pd.merge(particle_data, halo_particles, left_on = 'id', right_on = 'id', how = 'outer')
data['haloID'] = data['haloID'].fillna(0)
data['halo_mass'] = data['np'].fillna(0)
data = data[['id','x','y','z','vx','vy','vz','haloID','halo_mass']]

In [59]:
#converting the postion from kpc to Mpc
data['x'] = data['x']/1000
data['y'] = data['y']/1000
data['z'] = data['z']/1000
data = data.astype({'haloID':'int64'})
data = data.sort_values(by = 'x')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
data

,id,x,y,z,vx,vy,vz,haloID,halo_mass
2573056,3702803,0.104388,22.108690,36.533001,-1147.799561,-421.298645,-159.373352,0,0.0
2630144,4227097,0.106355,21.492632,40.919411,-1127.608154,-401.552826,-111.468544,0,0.0
11749888,10133520,0.107426,106.582535,96.523911,-1146.397827,135.866959,-227.624420,0,0.0
11750144,10133523,0.108411,107.197701,96.523987,-1141.314331,48.016113,-231.397110,0,0.0
2654464,4489219,0.108463,20.844633,42.775684,-1133.096680,-515.861816,-212.447449,0,0.0
...,...,...,...,...,...,...,...,...,...
10339839,15605748,159.840271,83.402657,148.985031,844.709229,-175.208862,-443.182190,915,5946.0
10348031,15605749,159.842987,83.417397,149.611725,859.852356,-94.854485,-464.136230,915,5946.0
10340351,15605740,159.844254,84.639664,149.009415,869.186951,-289.623627,-300.264465,915,5946.0
10348287,15605742,159.845825,84.053123,149.621384,885.247620,-69.581429,-391.406372,916,7980.0


In [61]:
box_size = 159
boundary = 20

d = data[data.x > boundary]

d = d[d.x < box_size - boundary]

d = d[d.y > boundary]

d = d[d.y < box_size - boundary]

d = d[d.z > boundary]

d = d[d.z < box_size - boundary]

In [62]:
d.shape

(6873748, 9)

In [63]:
from tqdm import tqdm
def box_mass(r_cutoff, data, sample):
    m = len(sample)
    mass = pd.DataFrame(np.zeros(m),columns=['den_contrast'])
    for i in tqdm(range(m)):
        wx=sample.iloc[i]['x']
        wy=sample.iloc[i]['y']
        wz=sample.iloc[i]['z']
        ix = np.searchsorted(data[:,2] , wx-r_cutoff)
        jx = np.searchsorted(data[:,2] , wx+r_cutoff)

        temp = np.where((data[ix:jx,3] < wy+r_cutoff) & (data[ix:jx,3] > wy-r_cutoff) & (data[ix:jx,4] < wz+r_cutoff) & (data[ix:jx,4] > wz-r_cutoff))
    
        mass.den_contrast[i] = len(temp[0])
    return mass

In [64]:
n = 100000

sample = d.sample(n, random_state = 42)
sample_density = sample[['id', 'haloID']]
sample_density.reset_index(drop = True, inplace = True)

In [65]:
sample_density.head()

,id,haloID
0,12445545,460
1,9757813,0
2,4364083,0
3,9111430,0
4,2698019,0


In [66]:
print(data)

                id           x           y  ...          vz  haloID  halo_mass
2573056    3702803    0.104388   22.108690  ... -159.373352       0        0.0
2630144    4227097    0.106355   21.492632  ... -111.468544       0        0.0
11749888  10133520    0.107426  106.582535  ... -227.624420       0        0.0
11750144  10133523    0.108411  107.197701  ... -231.397110       0        0.0
2654464    4489219    0.108463   20.844633  ... -212.447449       0        0.0
...            ...         ...         ...  ...         ...     ...        ...
10339839  15605748  159.840271   83.402657  ... -443.182190     915     5946.0
10348031  15605749  159.842987   83.417397  ... -464.136230     915     5946.0
10340351  15605740  159.844254   84.639664  ... -300.264465     915     5946.0
10348287  15605742  159.845825   84.053123  ... -391.406372     916     7980.0
10340095  15605743  159.846771   84.050972  ... -383.108978     915     5946.0

[16777216 rows x 9 columns]


In [67]:
data = data.to_numpy()

In [68]:
rc = np.array([])
for i in range(20, 30, 1):
  r =  0.6 * i + 0.4
  rc = np.append(rc, r)
  density = box_mass(r , data, sample)
  density = density.div((2*r)**3)
  sample_density = pd.concat([sample_density, density], axis = 1)
  sample_density = sample_density.rename(columns={'den_contrast': 'dc for {0:.1f}'.format(r)})
sample_density.head()

100%|██████████| 100000/100000 [40:17<00:00, 41.36it/s]


,id,haloID,dc for 12.4,dc for 13.0,dc for 13.6,dc for 14.2,dc for 14.8,dc for 15.4,dc for 16.0,dc for 16.6,dc for 17.2,dc for 17.8
0,12445545,460,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007689,0.007292
1,9757813,0,0.065233,0.069356,0.065893,0.063258,0.059651,0.057054,0.054413,0.051975,0.049794,0.047874
2,4364083,0,0.000000,0.000000,0.000000,0.000000,0.007056,0.007050,0.007751,0.008089,0.008033,0.008156
3,9111430,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2698019,0,3.437948,3.293411,3.157777,3.045101,2.937072,2.829307,2.735870,2.650381,2.567777,2.490225


In [69]:
sample_density.to_csv('/content/drive/MyDrive/sample_density_final.csv', index=False)

In [71]:
rho_bar = len(data) / (159.8**3)
sample_density.iloc[:,2:] -= rho_bar
sample_density.iloc[:,2:] /= rho_bar
print(sample_density)

             id  haloID  dc for 12.4  ...  dc for 16.6  dc for 17.2  dc for 17.8
0      12445545     460    -1.000000  ...    -1.000000    -0.998130    -0.998226
1       9757813       0    -0.984134  ...    -0.987358    -0.987889    -0.988356
2       4364083       0    -1.000000  ...    -0.998033    -0.998046    -0.998016
3       9111430       0    -1.000000  ...    -1.000000    -1.000000    -1.000000
4       2698019       0    -0.163801  ...    -0.355358    -0.375449    -0.394312
...         ...     ...          ...  ...          ...          ...          ...
99995   5727559       0    -0.993510  ...    -0.976757    -0.977379    -0.978048
99996  13779938       0    -0.995918  ...    -0.996098    -0.996027    -0.995892
99997   5313693       0    -0.980673  ...    -0.986321    -0.986873    -0.987472
99998   7482364       0    -1.000000  ...    -1.000000    -1.000000    -1.000000
99999   8333714       0    -0.046437  ...    -0.277434    -0.300888    -0.322414

[100000 rows x 12 columns]


In [72]:
sample_density.to_csv('/content/drive/MyDrive/density_contrast_final.csv', index=False)

The above gives the physical properties that we were looking for, namely the density contrasts for each particle encompassed by boxes of different sizes.